In [1]:
import pandas as pd
import numpy as np

### Load Dataset

In [2]:
# Load the Dataset
dataset_path = '../data/DS_ML Coding Challenge Dataset.xlsx'
train_dataset = pd.read_excel(dataset_path, sheet_name='Training Dataset')
test_dataset = pd.read_excel(dataset_path, sheet_name='Test Dataset')

In [3]:
train_dataset.shape

(550176, 8)

In [4]:
train_dataset

,ProductType,Manufacturer,Area Code,Sourcing Channel,Product Size,Product Type,Month of Sourcing,Sourcing Cost
0,NTM3,X1,A28,WHOLESALE,Large,Powder,2021-05-01,10.158
1,NTM2,X1,A9,DIRECT,Large,Powder,2020-10-01,134.281
2,NTM3,X2,A20,DIRECT,Large,Powder,2020-12-01,12.456
3,NTM3,X1,A18,WHOLESALE,Small,Powder,2021-02-01,107.220
4,NTM2,X1,A28,DIRECT,Large,Liquid,2020-11-01,197.763
...,...,...,...,...,...,...,...,...
550171,NTM2,X1,A5,DIRECT,Large,Powder,2020-07-01,136.469
550172,NTM3,X1,A14,DIRECT,Large,Liquid,2020-10-01,72.559
550173,NTM2,X1,A5,DIRECT,Small,Powder,2021-03-01,147.639
550174,NTM2,X1,A7,DIRECT,Small,Powder,2021-02-01,150.044


In [6]:
# Renaming columns
train_dataset.rename(columns={'ProductType':'ProductName'}, inplace=True)
train_dataset.columns = [column_name.replace(' ','') for column_name in train_dataset.columns]

# Renaming columns
test_dataset.rename(columns={'ProductType':'ProductName'}, inplace=True)
test_dataset.columns = [column_name.replace(' ','') for column_name in test_dataset.columns]

# Creating Combined Column
initial_column = 'ProductName'
prev_column = initial_column
column_order = ['AreaCode','Manufacturer','SourcingChannel','ProductSize','ProductType']

for column in column_order:
    initial_column = initial_column + '_' + column
    train_dataset[initial_column] = train_dataset[prev_column].map(str) + '_' + train_dataset[column]
    prev_column = initial_column

# Combined column name
column_name = 'ProductName_AreaCode_Manufacturer_SourcingChannel_ProductSize_ProductType'

# Grouping Products by CombinedKey
gb = train_dataset.groupby([column_name])
groups = [gb.get_group(group_name) for group_name in gb.groups]

new_train_dataset = pd.DataFrame()
for group in groups:
    df = group[[column_name,'SourcingCost','MonthofSourcing']].reset_index(drop=True)
    
    # Removing Outliers using Inter Quartile Range
    Q1 = np.percentile(df['SourcingCost'], 25, interpolation = 'midpoint') 
    Q3 = np.percentile(df['SourcingCost'], 75, interpolation = 'midpoint') 
    IQR = Q3 - Q1 
    old_shape = df.shape
    upper = np.where(df['SourcingCost'] > (Q3+1.5*IQR))
    lower = np.where(df['SourcingCost'] < (Q1-1.5*IQR))
    df.drop(upper[0], axis=0, inplace = True)
    df.drop(lower[0], axis=0, inplace = True)
    #print("Removed Outliers: ", old_shape[0]-df.shape[0])
    
    # Append to new dataframe
    new_train_dataset = new_train_dataset.append(df)
    
new_train_dataset[column_name.split('_')] = new_train_dataset[column_name].str.split('_',expand=True)
new_train_dataset.drop([column_name], axis=1, inplace=True)

In [7]:
new_train_dataset.shape

(511467, 8)